In [1]:
import json

# read similar_question_data.json

with open('similar_question_data.json') as f:
    data = json.load(f)

print(data[:5])

[{'question_id': '006d7', 'question_text': 'माना A = $\\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$, जहाँ i = √-1 है। तो समुच्चय { n ∈ {1,2,......,100} : $A^n$ = A} में अवयवों की संख्या है _______\n\nThe image shows a mathematical expression defining a matrix A and a set condition. The matrix A is a 2x2 matrix with elements 1+i, 1, -i, and 0, where i is defined as the square root of -1. The problem asks to find the number of elements in the set of n belonging to {1, 2, ..., 100} such that A to the power of n equals A.', 'subject': 'Mathematics', 'similar_questions': [{'similar_question_text': 'Let A = $\\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$ where I = $\\sqrt{-1}$. Then, the number of elements in the set {n$\\in$ {1,2, ....., 100} : $A^n$ = A} is', 'similarity_score': 0.981, 'summarized_solution_approach': 'The solution calculates $A^2$ and $A^4$. It finds that $A^4 = I$ (identity matrix). Consequently, $A^5 = A, A^9 = A$, and so on. The values of n for which $A^n = A$ 

In [2]:
len(data)

553

In [3]:
data[0]

{'question_id': '006d7',
 'question_text': 'माना A = $\\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$, जहाँ i = √-1 है। तो समुच्चय { n ∈ {1,2,......,100} : $A^n$ = A} में अवयवों की संख्या है _______\n\nThe image shows a mathematical expression defining a matrix A and a set condition. The matrix A is a 2x2 matrix with elements 1+i, 1, -i, and 0, where i is defined as the square root of -1. The problem asks to find the number of elements in the set of n belonging to {1, 2, ..., 100} such that A to the power of n equals A.',
 'subject': 'Mathematics',
 'similar_questions': [{'similar_question_text': 'Let A = $\\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$ where I = $\\sqrt{-1}$. Then, the number of elements in the set {n$\\in$ {1,2, ....., 100} : $A^n$ = A} is',
   'similarity_score': 0.981,
   'summarized_solution_approach': 'The solution calculates $A^2$ and $A^4$. It finds that $A^4 = I$ (identity matrix). Consequently, $A^5 = A, A^9 = A$, and so on. The values of n for which $A

In [4]:
data[0]["question_text"]

'माना A = $\\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$, जहाँ i = √-1 है। तो समुच्चय { n ∈ {1,2,......,100} : $A^n$ = A} में अवयवों की संख्या है _______\n\nThe image shows a mathematical expression defining a matrix A and a set condition. The matrix A is a 2x2 matrix with elements 1+i, 1, -i, and 0, where i is defined as the square root of -1. The problem asks to find the number of elements in the set of n belonging to {1, 2, ..., 100} such that A to the power of n equals A.'

In [5]:
data[0]["similar_questions"][0]

{'similar_question_text': 'Let A = $\\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$ where I = $\\sqrt{-1}$. Then, the number of elements in the set {n$\\in$ {1,2, ....., 100} : $A^n$ = A} is',
 'similarity_score': 0.981,
 'summarized_solution_approach': 'The solution calculates $A^2$ and $A^4$. It finds that $A^4 = I$ (identity matrix). Consequently, $A^5 = A, A^9 = A$, and so on. The values of n for which $A^n = A$ form an arithmetic progression: n = 1, 5, 9, ..., 97. The number of terms in this sequence is then calculated, which gives the number of elements in the set.'}

In [8]:
# basic qna logic


from google import genai
import os
from dotenv import load_dotenv
from google.genai import types
import json

from pydantic import BaseModel, ValidationError

class Solution(BaseModel):
    explanation: str
    final_answer: str

load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))


def get_raw_solution(question: str, max_retries: int = 3) -> Solution | None:
    """
    A straightfoward question answering function using the gemini model.
    Args:
        question (str): The question to be answered.
        max_retries (int): The maximum number of retries to get a response from the model (for object validation).
    Returns:
        dict: A dictionary containing the explanation and final answer.
    """
    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=f"""You are an academic expert at solving problems in the field of maths, physics and chemistry. 
            Respond with the solution to the given problem: {question}
            You respond with a JSON of explanation and final_answer where you can give step by step explanation in the explanation and the final solution in the final_answer.
            Keep final answer direct and as short as possible and keep the step by stem explanation to the explanation portion of the JSON
            """,
            config=types.GenerateContentConfig(
                thinking_config=types.ThinkingConfig(thinking_budget=0),
                response_mime_type="application/json",
                response_schema=Solution,
            ),
        )
        
        parsed = json.loads(response.text)
        solution = Solution(**parsed)
        return solution
        
    except (json.JSONDecodeError, ValidationError) as e:
        if max_retries > 0:
            return get_raw_solution(question, max_retries - 1)
        else:
            print("Max retries reached: JSON parse error.")
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None


get_raw_solution(data[0]["question_text"])


Solution(explanation="Let the given matrix be A = $\\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$. We need to find the number of integers n in the set {1, 2, ..., 100} such that $A^n = A$.\n\nFirst, let's calculate $A^2$:\n$A^2 = A \\cdot A = \\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix} \\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$\n$A^2 = \\begin{pmatrix} (1+i)(1+i) + 1(-i) & (1+i)(1) + 1(0) \\ -i(1+i) + 0(-i) & -i(1) + 0(0) \\end{pmatrix}$\n$A^2 = \\begin{pmatrix} (1 + 2i + i^2) - i & 1+i \\ (-i - i^2) & -i \\end{pmatrix}$\nSince $i^2 = -1$,\n$A^2 = \\begin{pmatrix} (1 + 2i - 1) - i & 1+i \\ (-i - (-1)) & -i \\end{pmatrix}$\n$A^2 = \\begin{pmatrix} 2i - i & 1+i \\ -i + 1 & -i \\end{pmatrix}$\n$A^2 = \\begin{pmatrix} i & 1+i \\ 1-i & -i \\end{pmatrix}$\n\nNow, let's calculate $A^3$:\n$A^3 = A^2 \\cdot A = \\begin{pmatrix} i & 1+i \\ 1-i & -i \\end{pmatrix} \\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$\n$A^3 = \\begin{pmatrix} i(1+i) + (1+i)(-i) & i(1) + (1+i)(0) \\ 

In [9]:
data[0]["similar_questions"]


[{'similar_question_text': 'Let A = $\\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$ where I = $\\sqrt{-1}$. Then, the number of elements in the set {n$\\in$ {1,2, ....., 100} : $A^n$ = A} is',
  'similarity_score': 0.981,
  'summarized_solution_approach': 'The solution calculates $A^2$ and $A^4$. It finds that $A^4 = I$ (identity matrix). Consequently, $A^5 = A, A^9 = A$, and so on. The values of n for which $A^n = A$ form an arithmetic progression: n = 1, 5, 9, ..., 97. The number of terms in this sequence is then calculated, which gives the number of elements in the set.'},
 {'similar_question_text': 'Let $$A = \\left( {\\matrix{\n   {1 + i} & 1  \\cr \n   { - i} & 0  \\cr \n\n } } \\right)$$ where $$i = \\sqrt { - 1} $$. Then, the number of elements in the set { n $$\\in$$ {1, 2, ......, 100} : A n = A } is ____________.',
  'similarity_score': 0.98,
  'summarized_solution_approach': 'First, the solution calculates A^2 and A^4. It finds that A^4 equals the identity matrix I. Con

In [10]:
# evaluation

class SimilarQuestion(BaseModel):
    similar_question_text: str
    similarity_score: float
    summarized_solution_approach: str
    
class SimilarQuestionsEvaluation(BaseModel):
    conceptual_similarity_score: int
    structural_similarity_score: int
    difficulty_alignment_score: int
    solution_approach_transferability_score: int
    
    
def evaluate_similar_question(question: str, similar_question: SimilarQuestion, max_retries: int = 3) -> SimilarQuestionsEvaluation:
    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=f"""You are an academic expert at analyzing and solving problems in the field of maths, physics and chemistry. 
            Here, you are given a question alongside a similar question, its solution approach and similarity score, and you are responsible for evaluating the similarity between the main question and it's similar question.
            You must assess how well the similar question represents the input question across the following dimensions.
            1. Conceptual Similarity: Do they test the same underlying concepts/principles?
            2. Structural Similarity: Are the problem structures analogous?
            3. Difficulty Alignment: Is the difficulty level appropriate?
            4. Solution Approach Transferability: Can the solution method be meaningfully applied?
            
            
            The input question is: {question}
            The similar question is: {str(similar_question.model_dump_json())}
            
            You respond with a JSON containing the following keys which are scores based on the aforementioned dimensions for analysis:
            
            conceptual_similarity_score: A score between 0 and 100
            structural_similarity_score: A score between 0 and 100
            difficulty_alignment_score: A score between 0 and 100 
            solution_approach_transferability_score: A score between 0 and 100
            
            Make sure to respond with a json strictly following the above format.
            
            """,
            config=types.GenerateContentConfig(
                thinking_config=types.ThinkingConfig(thinking_budget=0),
                response_mime_type="application/json",
                response_schema=SimilarQuestionsEvaluation,
            ),
        )
        
        parsed = json.loads(response.text)
        evaluation = SimilarQuestionsEvaluation(**parsed)
        return evaluation
        
    except (json.JSONDecodeError, ValidationError) as e:
        print("retrying")
        if max_retries > 0:
            return get_raw_solution(question, max_retries - 1)
        else:
            print("Max retries reached: JSON parse error.")
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None

    
        
evaluate_similar_question(data[0]["question_text"], SimilarQuestion(**data[0]["similar_questions"][0]))

SimilarQuestionsEvaluation(conceptual_similarity_score=100, structural_similarity_score=100, difficulty_alignment_score=100, solution_approach_transferability_score=100)

In [11]:
evaluate_similar_question(data[1]["question_text"], SimilarQuestion(**data[0]["similar_questions"][0]))

SimilarQuestionsEvaluation(conceptual_similarity_score=0, structural_similarity_score=0, difficulty_alignment_score=0, solution_approach_transferability_score=0)

In [12]:
data[0]["similar_questions"]

[{'similar_question_text': 'Let A = $\\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$ where I = $\\sqrt{-1}$. Then, the number of elements in the set {n$\\in$ {1,2, ....., 100} : $A^n$ = A} is',
  'similarity_score': 0.981,
  'summarized_solution_approach': 'The solution calculates $A^2$ and $A^4$. It finds that $A^4 = I$ (identity matrix). Consequently, $A^5 = A, A^9 = A$, and so on. The values of n for which $A^n = A$ form an arithmetic progression: n = 1, 5, 9, ..., 97. The number of terms in this sequence is then calculated, which gives the number of elements in the set.'},
 {'similar_question_text': 'Let $$A = \\left( {\\matrix{\n   {1 + i} & 1  \\cr \n   { - i} & 0  \\cr \n\n } } \\right)$$ where $$i = \\sqrt { - 1} $$. Then, the number of elements in the set { n $$\\in$$ {1, 2, ......, 100} : A n = A } is ____________.',
  'similarity_score': 0.98,
  'summarized_solution_approach': 'First, the solution calculates A^2 and A^4. It finds that A^4 equals the identity matrix I. Con

In [32]:
#basic solution building logic with llama index 

from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core.prompts import PromptTemplate
from llama_index.core.bridge.pydantic import BaseModel
from llama_index.llms.google_genai import GoogleGenAI
from google.genai import types


from typing import List

llm = GoogleGenAI(
    model="gemini-2.5-flash",
    api_key = os.getenv("GEMINI_API_KEY"),
    generation_config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            thinking_budget=0
        )  
    ),
)

class Solution(BaseModel):
    explanation: str
    final_answer: str


prompt_tmpl = PromptTemplate("""You are an academic expert at solving problems in the field of maths, physics and chemistry. 
            Respond with the solution to the given problem: {question}
            You respond with a JSON of explanation and final_answer where you can give step by step explanation in the explanation and the final solution in the final_answer.
            Keep final answer direct and as short as possible and keep the step by stem explanation to the explanation portion of the JSON
            """,)

solution_object = llm.as_structured_llm(Solution).complete(prompt_tmpl.format(question=data[0]["question_text"])).raw

print(solution_object)

explanation="The given matrix is A = $\x08egin{pmatrix} 1+i & 1 \\ -i & 0 \n\\end{pmatrix}$.\nWe need to find the number of n ∈ {1,2,......,100} such that $A^n$ = A.\nFirst, let's calculate $A^2$:\n$A^2 = A \\cdot A = \x08egin{pmatrix} 1+i & 1 \\ -i & 0 \n\\end{pmatrix} \x08egin{pmatrix} 1+i & 1 \\ -i & 0 \n\\end{pmatrix}$\n$A^2 = \x08egin{pmatrix} (1+i)(1+i) + 1(-i) & (1+i)(1) + 1(0) \\ (-i)(1+i) + 0(-i) & (-i)(1) + 0(0) \n\\end{pmatrix}$\n$A^2 = \x08egin{pmatrix} 1 + 2i + i^2 - i & 1+i \\ -i - i^2 & -i \n\\end{pmatrix}$\nSince $i^2 = -1$,\n$A^2 = \x08egin{pmatrix} 1 + 2i - 1 - i & 1+i \\ -i - (-1) & -i \n\\end{pmatrix}$\n$A^2 = \x08egin{pmatrix} i & 1+i \\ 1-i & -i \n\\end{pmatrix}$\nNow, let's calculate $A^3$:\n$A^3 = A^2 \\cdot A = \x08egin{pmatrix} i & 1+i \\ 1-i & -i \n\\end{pmatrix} \x08egin{pmatrix} 1+i & 1 \\ -i & 0 \n\\end{pmatrix}$\n$A^3 = \x08egin{pmatrix} i(1+i) + (1+i)(-i) & i(1) + (1+i)(0) \\ (1-i)(1+i) + (-i)(-i) & (1-i)(1) + (-i)(0) \n\\end{pmatrix}$\n$A^3 = \x08egin{p

In [27]:
solution_object.model_dump()["final_answer"]

'25'

In [29]:
data[0]

{'question_id': '006d7',
 'question_text': 'माना A = $\\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$, जहाँ i = √-1 है। तो समुच्चय { n ∈ {1,2,......,100} : $A^n$ = A} में अवयवों की संख्या है _______\n\nThe image shows a mathematical expression defining a matrix A and a set condition. The matrix A is a 2x2 matrix with elements 1+i, 1, -i, and 0, where i is defined as the square root of -1. The problem asks to find the number of elements in the set of n belonging to {1, 2, ..., 100} such that A to the power of n equals A.',
 'subject': 'Mathematics',
 'similar_questions': [{'similar_question_text': 'Let A = $\\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$ where I = $\\sqrt{-1}$. Then, the number of elements in the set {n$\\in$ {1,2, ....., 100} : $A^n$ = A} is',
   'similarity_score': 0.981,
   'summarized_solution_approach': 'The solution calculates $A^2$ and $A^4$. It finds that $A^4 = I$ (identity matrix). Consequently, $A^5 = A, A^9 = A$, and so on. The values of n for which $A

In [99]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent
from difflib import SequenceMatcher


class SimilarQuestion(BaseModel):
    similar_question_text: str
    similarity_score: float
    summarized_solution_approach: str

class SimilarQuestionsEvaluation(BaseModel):
    similar_question: str
    solution_approach: str
    conceptual_similarity_score: int
    structural_similarity_score: int
    difficulty_alignment_score: int
    solution_approach_transferability_score: int
    total_score: int
    
class SimilarQuestionsEvaluationList(BaseModel):
    original_question: str
    evaluations: List[SimilarQuestionsEvaluation]
    
    
def get_similar_questions(question: str) -> List[SimilarQuestion]:
    # ideally we do a vector similarity search here but rn just doing fuzzy matching with whatever question the llm passes to this function
    """ Get most similar question from dataset and return its similar questions """
    with open('similar_question_data.json') as f:
        data = json.load(f)

    best_match = max(
        data,
        key=lambda q: SequenceMatcher(None, q['question_text'], question).ratio()
    )

    return [SimilarQuestion(**sq) for sq in best_match.get('similar_questions', [])]

    
    
get_similar_questions_tool = FunctionTool.from_defaults(fn=get_similar_questions)

eval_prompt = """You are an academic expert at analyzing and solving problems in the field of maths, physics and chemistry. 
            Here, you are given a question and you must get similar questions to the question from the dataset and evaluate them and return the results.
            The image descriptions after the question given are also part of the question, make sure to include them.
            
            You must assess how well each similar question represents the input question across the following dimensions.
            
            1. Conceptual Similarity: Do they test the same underlying concepts/principles?
            2. Structural Similarity: Are the problem structures analogous?
            3. Difficulty Alignment: Is the difficulty level appropriate?
            4. Solution Approach Transferability: Can the solution method be meaningfully applied?
            
            You should get similar questions via the tool get_similar_questions_tool
            which accepts the question and responds with similar questions, their solution approaches and vector similarity scores.
            You must pass the question that you receive identically to the function without changing a single character.           
            
            You respond with a json with key original_question and another key called evaluations with its corresponding value which is a list of objects.
            Each object contain the following keys which are scores based on the aforementioned dimensions for analysis for each similar quesion:
            
            
            similar_question: The similar question
            solution_approach: The solution approach
            conceptual_similarity_score: A score between 0 and 100
            structural_similarity_score: A score between 0 and 100
            difficulty_alignment_score: A score between 0 and 100 
            solution_approach_transferability_score: A score between 0 and 100
            total_score: A score between 0 and 100 which is the average of all the scores
            
            Make sure to respond with a list of objects strictly following the above format.
            
            """



eval_agent = FunctionAgent(
    name="EvaluationAgent",
    description="It responds with similar questions and answers to a question alongside their solution approach and similarity scores",
    system_prompt=(
        eval_prompt
    ),
    llm=llm,
    tools=[get_similar_questions_tool],
    output_cls=SimilarQuestionsEvaluationList
)

response = await eval_agent.run(data[0]["question_text"])

In [100]:
data[0]["question_text"]

'माना A = $\\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$, जहाँ i = √-1 है। तो समुच्चय { n ∈ {1,2,......,100} : $A^n$ = A} में अवयवों की संख्या है _______\n\nThe image shows a mathematical expression defining a matrix A and a set condition. The matrix A is a 2x2 matrix with elements 1+i, 1, -i, and 0, where i is defined as the square root of -1. The problem asks to find the number of elements in the set of n belonging to {1, 2, ..., 100} such that A to the power of n equals A.'

In [101]:
response.structured_response

{'original_question': 'माना A = $\\begin{pmatrix} 1+i & 1 \\ -i & 0 \\end{pmatrix}$, जहाँ i = \n-1 है। तो समुच्चय { n \n {1,2,......,100} : $A^n$ = A} में अवयवों की संख्या है _______ \n\nThe image shows a mathematical expression defining a matrix A and a set condition. The matrix A is a 2x2 matrix with elements 1+i, 1, -i, and 0, where i is defined as the square root of -1. The problem asks to find the number of elements in the set of n belonging to {1, 2, ..., 100} such that A to the power of n equals A.',
 'evaluations': [{'similar_question': 'माना A = $\\begin{pmatrix} 2 & 2 \\ 2 & 2 \\end{pmatrix}$ तथा n \\u003e 1 एक पूर्णांक इस प्रकार है कि $A^n = kA$ है, तब k = __________ है।',
   'solution_approach': "The problem asks to find the value of k such that $A^n = kA$, where A is a given 2x2 matrix.First, calculate the powers of A: $A^2$, $A^3$, etc., to identify a pattern.Given A = $\\begin{pmatrix} 2 & 2 \\ 2 & 2 \\end{pmatrix}$.Calculate $A^2$: $A^2 = A \\cdot A = \\begin{pmatrix} 2

In [98]:
data[12]["question_text"]

"p. (1/y^2 * (cos(tan^{-1}y) + ysin(tan^{-1}y))/(cot(sin^{-1}y) + tan(sin^{-1}y)))^2 + y^4)^{1/2} का मान है\n\nThe image contains a mathematical expression. The expression involves trigonometric functions, inverse trigonometric functions, and algebraic terms. Specifically, it includes terms like cosine, sine, cotangent, and tangent, each applied to inverse trigonometric functions of 'y'. The expression also involves powers of 'y' and numerical constants. The entire expression is enclosed in parentheses and raised to the power of 1/2."

In [97]:
response = await eval_agent.run(data[12]["question_text"])
response.structured_response

{'original_question': 'p. (1/y^2 * (cos(tan^{-1}y) + ysin(tan^{-1}y))/(cot(sin^{-1}y) + tan(sin^{-1}y)))^2 + y^4)^{1/2} का मान है',
 'evaluations': [{'similar_question': '$$50\\\\tan \\\\left( {3{{\\\\tan }^{ - 1}}\\\\left( {{1 \\\\over 2}} \\\\right) + 2{{\\\\cos }^{ - 1}}\\\\left( {{1 \\\\over {\\\\sqrt 5 }}} \\\\right)} \\\\right) + 4\\\\sqrt 2 \\\\tan \\\\left( {{1 \\\\over 2}{{\\\\tan }^{ - 1}}(2\\\\sqrt 2 )} \\\\right)$$ is equal to ____________.',
   'solution_approach': 'The solution begins by simplifying the expression using trigonometric identities, specifically focusing on the tangent and inverse trigonometric functions. The expression is broken down into smaller, manageable parts. A key step involves recognizing that \\\\(2{{\\\\tan }^{ - 1}}\\\\left( {{1 \\nover 2}} \\\\right) + 2{{\\\\tan }^{ - 1}}(2) = \\\\pi \\\\). Then, the problem uses the half-angle formula for tangent to further simplify and solve for \\\\(\\\\tan \\\\alpha\\\\), where \\\\(\\\\alpha = {1 \\\\over 2

In [91]:
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'thought_signatures': [None], 'thoughts': '', 'tool_calls': []}, blocks=[TextBlock(block_type='text', text='{"evaluations": [{"similar_question": "$$50\\\\\\\\tan \\\\\\\\left( {3{{\\\\\\\\tan }^{ - 1}}\\\\\\\\left( {{1 \\\\\\\\over 2}} \\\\\\\\right) + 2{{\\\\\\\\cos }^{ - 1}}\\\\\\\\left( {{1 \\\\\\\\over {\\\\\\\\sqrt 5 }}} \\\\\\\\right)} \\\\\\\\right) + 4\\\\\\\\sqrt 2 \\\\\\\\tan \\\\\\\\left( {{1 \\\\\\\\over 2}{{\\\\\\\\tan }^{ - 1}}(2\\\\\\\\sqrt 2 )} \\\\\\\\right)$$ is equal to ____________.", "solution_approach": "The solution begins by simplifying the expression using trigonometric identities, specifically focusing on the tangent and inverse trigonometric functions. The expression is broken down into smaller, manageable parts. A key step involves recognizing that \\\\\\\\(2{{\\\\\\\\tan }^{ - 1}}\\\\\\\\left( {{1 \\\\nover 2}} \\\\\\\\right) + 2{{\\\\\\\\tan }^{ - 1}}(2) = \\\\\